This notebook shows the basic steps that are involved in movement detection. Much of what's in here can be encaspulated by higher-order functions or classes, but if you want to have full control over things, have a look at the following.

In [ ]:
import birdwatcher as bw

## Create a background subtractor object with suitable parameters

Create a background subtractor object with default parameters. This object basically encapsulates the algorithm that determines the background from a history of images, and uses that to find what is not background in the current image. See opencv page for more info on the algorithm: https://docs.opencv.org/3.4/d7/d7b/classcv_1_1BackgroundSubtractorMOG2.html

In [ ]:
bgs = bw.BackgroundSubtractorMOG2()

What are the parameters?

In [ ]:
bgs.get_params()

You can use non-default paramaters by specifying them at intstantiation.

In [ ]:
bgs = bw.BackgroundSubtractorMOG2(VarThreshold=70, NMixtures=8, History=3)

In [ ]:
bgs.get_params()

## Create a video object (see separate notebook on this)

In [ ]:
vf = bw.testvideosmall()

## Do movement detection by hand based on this background subtractor

This can be done by setting up a pipe line that generates and processes image sequences.

First set up a frame generator that produces gray frames from the color video file object:

In [ ]:
frames = vf.iter_frames(color=False)

Now feed that to the background subtractor that we created above. It returns another frame generator.

In [ ]:
frames = bgs.iter_apply(frames, learningRate=-1)

We now have a frame generator that produces foreground mask frames. Let's get rid of some noise. (look at MorphologyEx page of opencv for what this does: https://docs.opencv.org/3.4/d9/d61/tutorial_py_morphological_ops.html)

In [ ]:
frames = frames.morphologyex(morphtype='open', kernelsize=2)

We are ready to start running the whole frame pipe line and save results as a video for inspection.

In [ ]:
frames = frames.tovideo('output/test_MOG2.mp4', framerate=vf.avgframerate)

This whole pipeline can also be shortened like so:

In [ ]:
frames = (bgs.iter_apply(vf.iter_frames(color=False), learningRate=-1)
                        .morphologyex(morphtype='open', kernelsize=2)
                        .tovideo('output/test_MOG2.mp4', framerate=vf.avgframerate))

## Create a coordinate array for storage of the results

We wrote the movement detection results (suprathreshold pixels) above to a video so that we could view the results. However, if you want to save the results for further analyses it is much better to save tham as a *coordinate array*.

A coordinate array really is just a Darr ragged array (see separate library). This makes it easy to read the data in other environments, e.g. R. We save some metadata so that we later know what we did. Instead of saving the frames to video, we now detect non-zero pixel (i.e. foreground) coordinates, and save that to the coordinate array.

First set up analysis pipe line:

In [ ]:
coordinates = (bgs.iter_apply(vf.iter_frames(color=False), learningRate=-1)
                             .morphologyex(morphtype='open', kernelsize=2)
                             .find_nonzero())

Create an empty CoordinateArray object to save to:

In [ ]:
coordsarray = bw.create_coordarray('output/testcoords.darr', 
                                   framewidth=vf.framewidth,
                                   frameheight=vf.frameheight,
                                   metadata={'bgsparams': bgs.get_params(),
                                             'morphologyex': ('open', 2),
                                             'learningrate': -1}, 
                                   overwrite=True)

In [ ]:
coordsarray.metadata

Now save pipeline to array:

In [ ]:
%%time
for c in coordinates:
    coordsarray.append(c)

## Accessing coordinate arrays

The coordinate array can be accessed in other python sessions like so:

In [ ]:
coordsarray = bw.CoordinateArrays('output/testcoords.darr')

In [ ]:
coordsarray[100] # coordinates of the 101th frame 

## Take it together and look at a range of parameters

As an example we'll look at a range of history settings. Note that we do not have to run the analysis pipe line twice in order to get both coordinate results and a video. We just create a coordinate array first. This can then be saved as a video for inspection.

In [ ]:
%%time
vf = bw.testvideosmall()
for history in (2,3,4):
    bgs = bw.BackgroundSubtractorMOG2(History=history, VarThreshold=50)
    coordinates = (bgs.iter_apply(vf.iter_frames(color=False), learningRate=-1)
                         .morphologyex(morphtype='open', kernelsize=2)
                         .find_nonzero())
    basefilename = f'testcoords_hist{history}'
    coordsarray = bw.create_coordarray(f'output/{basefilename}.darr', 
                                       framewidth=vf.framewidth,
                                       frameheight=vf.frameheight, 
                                       metadata={'bgsparams': bgs.get_params(),
                                                 'morphologyex': ('open', 2),
                                                 'learningrate': -1}, 
                                       overwrite=True)
    for c in coordinates:
        coordsarray.append(c)
    coordsarray.tovideo(f'output/{basefilename}.mp4', framerate=vf.avgframerate)